# Estimation of primary magma composition by decoding crustal magma processes based on Bayesian statistical analysis of chemical zoning of phenocrysts in volcanic rocks


## Method

### Parallel Tempering Markov Chain Monte-Carlo (PT-MCMC) method

#### MCMC method

* Algorithm of **random sampling from any probability distribution**
* Frequency of sampled parameters are proportional to their posterior probability
* We can evaluate stochastic property of parameters (standered error _etc._)


Based on Bayes' theory: Update prior probability $P(\theta)$ to posterior probability $P(x|\theta)$ by likelihood function $L(x|\theta)$ using information of data,  


$$
    P(x|\theta) \propto L(x|\theta)P(\theta)
$$


| Symbol |  |
|:--- |:---|
|$x$ | Data |
|$\theta$ | Model parameters |
|$P(\theta)$ | Prior probability |
|$P(x|\theta)$ | Posterior probability |
|$L(x|\theta)$ | Likelihood function |

#### Example of likelihood function

* Normal distribution (Gaussian distribution)

$$
L(x) = \frac{1}{\sqrt{2\pi}\sigma} \exp\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)
$$

| Symbol | |
|:---: |---:|
|$\mu$ | Mean |
|$\sigma$| Standerd deviation |

* Occurence probability of data from model where the error of data is Gaussian

$$
    L(x|\theta) = \prod \frac{1}{\sqrt{2\pi}\epsilon_i} \exp\left(-\frac{\left(\hat{x_i(\theta)}-x_i\right)^2}{2\epsilon_i^2}\right)
$$

|Symbol ||
|:---:|---:|
|$x_i$ | $i$th data |
|$\theta$|model parameters |
|$\hat{x_i(\theta)}$ |Modeled value for $i$th data|
|$\epsilon_i$ |Error of $i$th data|

#### Parallel tempering (or replica exchanging) storategy

* For escaping from local solution
* For efficient sampling

Extanding MCMC method by adding hyper parameter of **"temperature of probability distribution"** $T_\beta$,

$$
    P(x|\theta,T_\beta) \propto P(x|\theta)^{1/T_\beta}
$$

The $T_\beta$ has effect of flattening probability distribution.

|||
|:---:|:---:|
|low $T_\beta$| Dense sampling around local solution |
|high $T_\beta$| Sampling from vast parameters space |

By creating many Markov chains having different temperature and exchanging the temperatures among them, 
we can realize dense sampling around various local solutions including the global solusion.

In [1]:
var tf = require("../../jslib/textParser")
var Plotly = require("../node_modules/ijavascript-plotly")

In [2]:
var range = (ini,fin,step=1) => [...Array(Math.ceil((fin-ini)/step))].map((_,i)=>ini+step*i)
var getZ = (f,r) => {
    const diff = r.map((_,i)=>(i === 0)
                    ? 0
                    : r[i]-r[i-1]
                )
    return r.map((x,i)=>f(x)*diff[i]).reduce((a,b)=>a+b,0)
}

var Gauss = (mu,sigma) => x => 1/(Math.sqrt(2*Math.PI)*sigma) * Math.exp(-0.5*(x-mu)**2/sigma**2)


### Demo: direct sampling from probability distribution

* Random sampling from mixed normal distribution: 


$$
   P(x|\theta) \propto L(x) = \sum \frac{1}{\sqrt{2\pi}\sigma_i} \exp\left(-\frac{(x-\mu_i)^2}{2\sigma_i^2}\right)
$$

| Symbol | |
|:---: |---:|
|$\mu$ | Mean |
|$\sigma$| Standerd deviation |

In [3]:
var g1 = Gauss(1,1);
var g2 = Gauss(5,2);
var g3 = Gauss(-5,1.5);
var mixedGauss = x => g1(x) + g2(x) + g3(x)
var z_mixedGauss = getZ(mixedGauss,range(-10,10,0.01))

Plotly(
    [
        {
            x : range(-10,10,0.1),
            y : range(-10,10,0.1).map(x=>g1(x)),
            line : {
                dash : "dash"
            }
        },
        {
            x : range(-10,10,0.1),
            y : range(-10,10,0.1).map(x=>g2(x)),
            line : {
                dash : "dash"
            }
        },
        {
            x : range(-10,10,0.1),
            y : range(-10,10,0.1).map(x=>g3(x)),
            line : {
                dash : "dash"
            }
        },
        {
            x : range(-10,10,0.1),
            y : range(-10,10,0.1).map(x=>mixedGauss(x))
        }
    ],
    {
        title : "Mixed normal distribution"
    }
)

#### Sample multiple $x$ randomly form mixed normal distribution

* By naive MCMC method
* By PT-MCMC method with 16 Markov chains

In [5]:
var gauss_oneMCMC = tf.read_csv("../result/demo1/sample-0-2018_0723_135041.csv")
var gauss_ptMCMC = range(0,16).map(n=>tf.read_csv(`../result/demo1/sample-${n}-2018_0723_141245.csv`));

In [6]:
Plotly(
    [
        {
            y : gauss_oneMCMC.x0,
            opacity : 0.75,
            name : "by naive MCMC"
        },
        {
            y : gauss_ptMCMC[0].x0,
            opacity : 0.75,
            name : "by PT-MCMC"
        }
    ],
    {
        title : "Transition of sampled parameter",
        xaxis : {
            title : "Sampling time"
        },
        yaxis : {
            title : "x"
        }
    }
)

In [7]:
Plotly(
    [
        {
            x: gauss_oneMCMC.x0,
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.5,
            name : "by naive MCMC"
        },
        {
            x : gauss_ptMCMC[0].x0,
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.55,
            name : "by PT-MCMC"
        },
        {
            x : range(-10,10,0.1),
            y : range(-10,10,0.1).map(x=>mixedGauss(x)/z_mixedGauss),
            name : "Ideal distribution",
            line : {
                color : "red"
            }
        }
    ],
    {
        barmode: "overlay",
        title : "1000 samples from mixed normal distribution",
        yaxis : {
            title : "Probability density"
        }
    }
)

#### Why PT-MCMC method can be escaped from local solution ?

In [9]:
Plotly(
    gauss_ptMCMC.map(df=>{
        return {
            x: df.x0,
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.25
        }
    }),
    {
        barmode: "overlay",
        title : "Effect of tempering",
        xaxis : {
            range : [-25,25]
        }
    }
)

### Demo: Parameter sampling in fitting $\tanh$ to data

The fitting model is,

$$
f(x|\theta) = a \tanh\frac{x}{b} + c
$$

The data is 

$$
    y(x) = 10 \tanh\frac{x}{2} - 5 + \epsilon_y
$$

$$
    \epsilon_x = 0
$$

$$
    \epsilon_y = N(0,2)
$$

The likelihood function is occurence probability of the data.

The posterior probability of the parameters $\theta = (a,b,c)$ are sampled.

2018_0723_154403では, PTMCMCのswapChainメソッドに誤りがあった. 

2018_0726_171317ではそれが修正されている.

両者は同じ乱数列で実行されているが, 得られている結果が少し異なっている.

In [13]:
var tanh = range(0,16).map(n=>tf.read_csv(`../result/demo2/sample-${n}-2018_0723_154403.csv`));
var tanh2 = range(0,16).map(n=>tf.read_csv(`../result/demo2/sample-${n}-2018_0726_171317.csv`));
var tanhFunc = (a,b,c) => x => a*Math.tanh(x/b) + c

In [18]:
Plotly(
    [
        {
            y : tanh2[0].a0,
            name : "a"
        },
        {
            y : tanh2[0].b0,
            name : "b"
        },
        {
            y : tanh2[0].c0,
            name : "c"
        }
    ],
    {
        title : "Transition of sampled parameters"
    }
)

In [16]:
Plotly(
    [
        {
            x : tanh[0].a0,
            y : tanh2[0].a0,
            name : "a",
            mode:"markers"
        },
    ]
)

In [17]:
Plotly(
    [
        {
            x : tanh2[0].a0,
            name : "a",
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.5
        },
        {
            x : tanh2[0].b0,
            name : "b",
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.5
        },
        {
            x : tanh2[0].c0,
            name : "c",
            type : "histogram",
            histnorm: 'probability',
            opacity : 0.5
        }
    ],
    {
        barmode: "overlay"
    }
)

* Histogram of $c$ is unimodal
* $a$ and $b$ are bimodal

In [20]:
Plotly(
    [
        {
            x : tanh2[0].a0,
            y : tanh2[0].b0,
            mode : "markers"
        }
    ],
    {
        width : 600,
        height : 600,
        title : "Combination of parameter a and b",
        xaxis : {
            title: "a"
        },
        yaxis : {
            title : "b"
        }
    }
)

* Trade-off between $a$ and $b$ is shown
* There are two cluster of optimum combination of $a$ and $b$

Two clusters of optimized parameters are obtained 

because we have no information on prior probability distribution of $a$ or $b$. 

In [21]:
Plotly(
    [
        {
            x : range(-1,20),
            y : range(-1,20).map(x=>tanhFunc(9,1,-5)(x)),
            name : "a = 9, b = 1"
        },
        {
            x : range(-1,20),
            y : range(-1,20).map(x=>tanhFunc(-9,-1,-5)(x)),
            name : "a = -9, b = -1"
        }
    ],
    {
        width : 600,
        height : 600,
        title : "Two cluster of optimized parameters",
        xaxis : {
            title: "x"
        },
        yaxis : {
            title : "y"
        }
    }
)

----

## Model of generating chemical profile in a grain

### Overview

We consider **multiple stages of crustal magma processes**

* crsyatl growth
* Elemental diffusion in grains
* Magma mixing 

```
Primary melt
    -> crystal growth       |
    -> elemental diffusion  | Stage1
    -> magma mixing         |
    
    -> crystal growth       |
    -> elemental diffusion  | Stage 2
    -> magma mixing         |
    
    ...
    
    -> errupted magma
```